##### Copyright 2019 The TensorFlow Authors.


In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Keras 的分布式训练

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/tutorials/distribute/keras"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a>   </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/distribute/keras.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 上运行</a>   </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/distribute/keras.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a>   </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tutorials/distribute/keras.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a>   </td>
</table>

## 概述

`tf.distribute.Strategy` API 提供了一个抽象，用于跨多个处理单元进行分布式训练。它允许您使用现有模型和训练代码，只需要很少的修改，就可以执行分布式训练。

本教程演示了如何使用 `tf.distribute.MirroredStrategy` 在*单台机器的多个 GPU 上通过同步训练*进行计算图内复制。该策略本质上是将所有模型变量复制到每个处理器。 然后，通过使用[全归约](http://mpitutorial.com/tutorials/mpi-reduce-and-allreduce/)来组合所有处理器的梯度，并将组合后的值应用于模型的所有副本。

您将使用 `tf.keras` API 构建模型并使用 `Model.fit` 对其进行训练。（要了解使用自定义训练循环和 `MirroredStrategy` 的分布式训练，请查看[此教程](custom_training.ipynb)。）

`MirroredStrategy` 在单台机器上的多个 GPU 上训练您的模型。要*在多个工作进程的多个 GPU 上进行同步训练*，请通过 [Keras Model.fit](multi_worker_with_keras.ipynb) 或[自定义训练循环](multi_worker_with_ctl.ipynb)使用 `tf.distribute.MultiWorkerMirroredStrategy`。有关其他选项，请参阅[分布式训练指南](../../guide/distributed_training.ipynb)。

要了解其他各种策略，请参阅[使用 TensorFlow 进行分布式训练](../../guide/distributed_training.ipynb)指南。

## 安装

In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf

import os

# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [3]:
print(tf.__version__)

2.19.0


## 下载数据集

从 [TensorFlow Datasets](https://tensorflow.google.cn/datasets) 加载 MNIST 数据集。这将返回 `tf.data` 格式的数据集。

将 `with_info` 参数设置为 `True` 会包含整个数据集的元数据，这些元数据将被保存到 `info` 中。此外，该元数据对象还包括训练样本和测试样本的数量。

In [4]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = datasets['train'], datasets['test']

/opt/anaconda3/envs/TensorFlow/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:  75%|███████▌  | 3/4 [00:02<00:00,  1.42 url/s]

Dl Completed...: 100%|██████████| 4/4 [00:05<00:00,  1.30s/ url]
                                                                        

Dataset mnist downloaded and prepared to /Users/bilibili/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


## 定义分布式策略

创建 `MirroredStrategy` 对象。这将处理分布，并提供一个上下文管理器 (`MirroredStrategy.scope`) 在内部构建模型。

In [5]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [6]:
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 1


## 设置输入流水线

当使用多个 GPU 训练模型时，可以通过增加批次大小来有效利用额外的计算能力。通常，应使用适合 GPU 内存的最大批次大小，并相应地调整学习率。

In [7]:
# You can also do info.splits.total_num_examples to get the total
# number of examples in the dataset.

num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

定义一个函数，将图像像素值从 `[0, 255]` 范围归一化到 `[0, 1]` 范围（[特征缩放](https://en.wikipedia.org/wiki/Feature_scaling)）：

In [8]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255

  return image, label

将此 `scale` 函数应用于训练数据和测试数据，使用 `tf.data.Dataset` API 对训练数据进行乱序 (`Dataset.shuffle`)，然后进行分批 (`Dataset.batch`)。请注意，您还保留了训练数据的内存缓存以提高性能 (`Dataset.cache`).。

In [9]:
train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

## 生成模型

在 `Strategy.scope` 的上下文中，使用 Keras API 创建和编译模型：

In [10]:
with strategy.scope():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10)
  ])

  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

/opt/anaconda3/envs/TensorFlow/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## 定义回调（callback）


定义以下 [Keras 回调](https://tensorflow.google.cn/guide/keras/train_and_evaluate)：

- `tf.keras.callbacks.TensorBoard`：为 TensorBoard 编写日志，以便呈现计算图。
- `tf.keras.callbacks.ModelCheckpoint`：以特定频率保存模型，例如在每个周期之后。
- `tf.keras.callbacks.BackupAndRestore`：通过备份模型和当前周期号来提供容错功能。在[使用 Keras 进行多工作进程训练](multi_worker_with_keras.ipynb)教程的*容错*部分了解详情。
- `tf.keras.callbacks.LearningRateScheduler`: schedules the learning rate to change after, for example, every epoch/batch.

出于说明目的，添加名为 <code>PrintLR</code> 的回调以在笔记本中显示学习率。

**注：** 使用 `BackupAndRestore` 回调而不是 `ModelCheckpoint` 作为从作业失败重新启动时还原训练状态的主要机制。由于 `BackupAndRestore` 仅支持 Eager 模式，在计算图模式下考虑使用 `ModelCheckpoint`。

In [11]:
# Define the checkpoint directory to store the checkpoints.
checkpoint_dir = './training_checkpoints'
# Define the name of the checkpoint files.
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

In [12]:
# Define a function for decaying the learning rate.
# You can define any decay function you need.
def decay(epoch):
  if epoch < 3:
    return 1e-3
  elif epoch >= 3 and epoch < 7:
    return 1e-4
  else:
    return 1e-5

In [13]:
# Define a callback for printing the learning rate at the end of each epoch.
class PrintLR(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    print('\nLearning rate for epoch {} is {}'.format(        epoch + 1, model.optimizer.lr.numpy()))

In [14]:
# Put all the callbacks together.
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                       save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay),
    PrintLR()
]

ValueError: When using `save_weights_only=True` in `ModelCheckpoint`, the filepath provided must end in `.weights.h5` (Keras weights format). Received: filepath=./training_checkpoints/ckpt_{epoch}

## 训练和评估

现在，以普通方式训练模型，在模型上调用 Keras `Model.fit` 并传入在教程开始时创建的数据集。无论您是否分布训练，此步骤相同。

In [ ]:
EPOCHS = 12

model.fit(train_dataset, epochs=EPOCHS, callbacks=callbacks)

2023-11-07 23:34:25.196020: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:553] The `assert_cardinality` transformation is currently not handled by the auto-shard rewrite and will be removed.


Epoch 1/12


INFO:tensorflow:Collective all_reduce tensors: 6 all_reduces, num_devices = 4, group_size = 4, implementation = CommunicationImplementation.NCCL, num_packs = 1


INFO:tensorflow:Collective all_reduce tensors: 6 all_reduces, num_devices = 4, group_size = 4, implementation = CommunicationImplementation.NCCL, num_packs = 1


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...


209/235 [=========================>....] - ETA: 0s - loss: 0.0388 - accuracy: 0.9899


217/235 [==========================>...] - ETA: 0s - loss: 0.0387 - accuracy: 0.9901


225/235 [===========================>..] - ETA: 0s - loss: 0.0385 - accuracy: 0.9901


232/235 [============================>.] - ETA: 0s - loss: 0.0387 - accuracy: 0.9900


Learning rate for epoch 12 is 9.999999747378752e-06

235/235 [==============================] - 2s 7ms/step - loss: 0.0388 - accuracy: 0.9900 - lr: 1.0000e-05


查看保存的检查点：

In [ ]:
# Check the checkpoint directory.
!ls {checkpoint_dir}

checkpoint		     ckpt_4.data-00000-of-00001
ckpt_1.data-00000-of-00001   ckpt_4.index
ckpt_1.index		     ckpt_5.data-00000-of-00001
ckpt_10.data-00000-of-00001  ckpt_5.index
ckpt_10.index		     ckpt_6.data-00000-of-00001
ckpt_11.data-00000-of-00001  ckpt_6.index
ckpt_11.index		     ckpt_7.data-00000-of-00001
ckpt_12.data-00000-of-00001  ckpt_7.index
ckpt_12.index		     ckpt_8.data-00000-of-00001
ckpt_2.data-00000-of-00001   ckpt_8.index
ckpt_2.index		     ckpt_9.data-00000-of-00001
ckpt_3.data-00000-of-00001   ckpt_9.index
ckpt_3.index


要查看模型的执行情况，请加载最新的检查点，并在测试数据上调用 `Model.evaluate`：

In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

eval_loss, eval_acc = model.evaluate(eval_dataset)

print('Eval loss: {}, Eval accuracy: {}'.format(eval_loss, eval_acc))

2023-11-07 23:34:58.383301: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:553] The `assert_cardinality` transformation is currently not handled by the auto-shard rewrite and will be removed.



 1/40 [..............................] - ETA: 57s - loss: 0.0627 - accuracy: 0.9805


 9/40 [=====>........................] - ETA: 0s - loss: 0.0467 - accuracy: 0.9848 


17/40 [===========>..................] - ETA: 0s - loss: 0.0523 - accuracy: 0.9823


25/40 [=================>............] - ETA: 0s - loss: 0.0507 - accuracy: 0.9827


33/40 [=======================>......] - ETA: 0s - loss: 0.0506 - accuracy: 0.9827


40/40 [==============================] - ETA: 0s - loss: 0.0501 - accuracy: 0.9831


40/40 [==============================] - 2s 8ms/step - loss: 0.0501 - accuracy: 0.9831


Eval loss: 0.05011461675167084, Eval accuracy: 0.9830999970436096


要可视化输出，请启动 TensorBoard 并查看日志：

In [ ]:
%tensorboard --logdir=logs

<!-- <img class="tfo-display-only-on-site" src="images/tensorboard_distributed_training_with_keras.png"/> -->

In [ ]:
!ls -sh ./logs

total 4.0K
4.0K train


## 保存模型

使用 `Model.save` 将模型保存到一个 `.keras` 压缩归档中。保存后，您可以使用或不使用 `Strategy.scope` 加载模型。

In [ ]:
path = 'my_model.keras'

In [ ]:
model.save(path)

现在，在没有 `Strategy.scope` 的情况下加载模型：

In [ ]:
unreplicated_model = tf.keras.models.load_model(path)

unreplicated_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy'])

eval_loss, eval_acc = unreplicated_model.evaluate(eval_dataset)

print('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))


 1/40 [..............................] - ETA: 12s - loss: 0.0627 - accuracy: 0.9805


17/40 [===========>..................] - ETA: 0s - loss: 0.0523 - accuracy: 0.9823 


32/40 [=======================>......] - ETA: 0s - loss: 0.0508 - accuracy: 0.9825


40/40 [==============================] - 0s 3ms/step - loss: 0.0501 - accuracy: 0.9831


Eval loss: 0.05011461302638054, Eval Accuracy: 0.9830999970436096


使用 `Strategy.scope` 加载模型：

In [ ]:
with strategy.scope():
  replicated_model = tf.keras.models.load_model(path)
  replicated_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                           optimizer=tf.keras.optimizers.Adam(),
                           metrics=['accuracy'])

  eval_loss, eval_acc = replicated_model.evaluate(eval_dataset)
  print ('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

2023-11-07 23:35:01.980132: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:553] The `assert_cardinality` transformation is currently not handled by the auto-shard rewrite and will be removed.



 1/40 [..............................] - ETA: 1:10 - loss: 0.0627 - accuracy: 0.9805


11/40 [=======>......................] - ETA: 0s - loss: 0.0508 - accuracy: 0.9830  


22/40 [===============>..............] - ETA: 0s - loss: 0.0500 - accuracy: 0.9833


33/40 [=======================>......] - ETA: 0s - loss: 0.0506 - accuracy: 0.9827


40/40 [==============================] - 2s 5ms/step - loss: 0.0501 - accuracy: 0.9831


Eval loss: 0.05011461675167084, Eval Accuracy: 0.9830999970436096


### 其他资源

更多通过 Keras `Model.fit` API 使用不同分布策略的示例：

1. [在 TPU 上使用 BERT 解决 GLUE 任务](https://tensorflow.google.cn/text/tutorials/bert_glue)教程使用 `tf.distribute.MirroredStrategy` 在 GPU 上进行训练，并使用 `tf.distribute.TPUStrategy` 在 TPU 上进行训练。
2. [使用分布式策略保存和加载模型](save_and_load.ipynb)教程演示了如何将 SavedModel API 与 `tf.distribute.Strategy` 一起使用。
3. [官方 TensorFlow 模型](https://github.com/tensorflow/models/tree/master/official)可以配置为运行多个分布式策略。

要了解有关 TensorFlow 分布式策略的更多信息，请参阅以下资料：

1. [使用 tf.distribute.Strategy 进行自定义训练](custom_training.ipynb)教程展示了如何使用 `tf.distribute.MirroredStrategy` 通过自定义训练循环进行单工作进程训练。
2. [使用 Keras 进行多工作进程训练](multi_worker_with_keras.ipynb)教程展示了如何将 `MultiWorkerMirroredStrategy` 与 `Model.fit` 一起使用。
3. [使用 Keras 和 MultiWorkerMirroredStrategy 的自定义训练循环](multi_worker_with_ctl.ipynb)教程展示了如何将 `MultiWorkerMirroredStrategy` 与 Keras 和自定义训练循环一起使用。
4. [TensorFlow 中的分布式训练](https://tensorflow.google.cn/guide/distributed_training)指南概述了可用的分布式策略。
5. [使用 tf.function 获得更佳性能](../../guide/function.ipynb)指南提供了有关其他策略和工具的信息，例如可用于优化 TensorFlow 模型性能的 [TensorFlow Profiler](../../guide/profiler.md)。

注：`tf.distribute.Strategy` 正在积极开发中，TensorFlow 将在不久的将来添加更多示例和教程。请进行尝试。我们欢迎您通过 [GitHub 上的议题](https://github.com/tensorflow/tensorflow/issues/new)提交反馈。